In [ ]:
import

### Notes
In English tweets for Ukraine 4/17/2022, the Firend Count of 0.1 quantile is 22 and the Follower Count of 0.1 quantile is 6. To remove potential fake accounts, use the tweets from uses of more than 20 Fridens, 5 Followers, and with some acctdesc. df[~((df['Friends Count'] <= 20)&( df['Follower Count'] <= 5)& df['acctdesc'].isnull())]

In [ ]:
# Rename and drop columns 
tweets_us_north = (tweets_us_north
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )